In [ ]:
# def fmri_preprocessing():
#     """
#     conducts remaining preprocessing steps on the fmris
#     :return: saves the preprocessed files for later use
#     """
#     import pickle
#     def load_dict(filename_):
#         with open(filename_, 'rb') as f:
#             u = pickle._Unpickler(f)
#             u.encoding = 'latin1'
#             ret_di = u.load()
#         return ret_di
# 
#     # path to ROI file
#     ROI_file = "participants_data_v2021/full_track/sub04/WB.pkl"
# 
#     # loading .pkl file
#     ROI_data = load_dict(ROI_file)
#     print(ROI_data.keys())
# 
#     # print the data dimensions:
#     print(ROI_data['train'].shape)
#     print(ROI_data['voxel_mask'].shape)
# 
#     # data shape: (1000, 3, 19445) - three measurements for 1000 videos of subject4, who has 19445 voxels
#     # mask shape: voxels are organized in a space of (78x93x71) voxels
# 
#     print("This has not be implemented yet.")
# 
# # voxel correlations (taken from CCN2021_Algonauts)
# def vectorized_correlation(x,y):
#     dim = 0
# 
#     centered_x = x - x.mean(axis=dim, keepdims=True)
#     centered_y = y - y.mean(axis=dim, keepdims=True)
# 
#     covariance = (centered_x * centered_y).sum(axis=dim, keepdims=True)
# 
#     covariance = covariance / (x.shape[dim])
# 
#     x_std = x.std(axis=dim, keepdims=True)+1e-8
#     y_std = y.std(axis=dim, keepdims=True)+1e-8
# 
#     corr = covariance / (x_std * y_std)
# 
#     return corr.ravel()

In [1]:
# shapes of fmri scans: differ a lot, i.e. sub2 has double the size of sub7. Because of that, combining the scans to a common label does not seem feasible
import pickle

# Load the object from the PKL file
for sub in ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]:
    with open(f'./participants_data_v2021/full_track/{sub}/WB.pkl', 'rb') as file:
        loaded_object = pickle.load(file)
    
    voxel_data = loaded_object.get("train")
    print(f"Shape of fmri data for {sub}:", voxel_data.shape)


Shape of fmri data for sub01: (1000, 3, 18222)
Shape of fmri data for sub02: (1000, 3, 21573)
Shape of fmri data for sub03: (1000, 3, 15225)
Shape of fmri data for sub04: (1000, 3, 19445)
Shape of fmri data for sub05: (1000, 3, 13340)
Shape of fmri data for sub06: (1000, 3, 19818)
Shape of fmri data for sub07: (1000, 3, 10836)
Shape of fmri data for sub08: (1000, 3, 12347)
Shape of fmri data for sub09: (1000, 3, 17570)
Shape of fmri data for sub10: (1000, 3, 12950)


In [11]:
import os
import numpy as np
import pandas as pd
import pickle

In [21]:
# get shapes of ROI data per subject

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        ret_di = u.load()
    return ret_di

def get_fmri(sub, ROI, avg=True):
    """
    Loads fMRI data for a specified ROI into a numpy array.

    Parameters:
    fmri_dir (str): Path to fMRI data.
    ROI (str): Name of the ROI.
    avg (bool, optional): If True, average data across repetitions. Defaults to True.

    Returns:
    np.array: fMRI response matrix (dimensions: #train_vids x #repetitions x #voxels).
    (Optional) np.array: Voxel mask for the 'WB' ROI.
    """
    fmri_dir = './participants_data_v2021'
    if ROI == "WB":
        track = "full_track"
    else:
        track = "mini_track"
    track_dir = os.path.join(fmri_dir, track)
    sub_fmri_dir = os.path.join(track_dir, sub)
    
    ROI_data = load_dict(os.path.join(sub_fmri_dir, f"{ROI}.pkl"))
    data = np.mean(ROI_data["train"], axis=1) if avg else ROI_data["train"]
    return (data, ROI_data['voxel_mask']) if ROI == "WB" else data

In [3]:
os.getcwd()

'C:\\Users\\marce\\PycharmProjects\\Brainvision_Project'

In [22]:
# Subjects and ROIs
subs = ["sub01", "sub02", "sub03", "sub04", "sub05", "sub06", "sub07", "sub08", "sub09", "sub10"]
ROIs = ["V1", "V2", "V3", "V4", "LOC", "EBA", "FFA", "STS", "PPA", "WB"]

# Initialize a DataFrame to store shapes
shape_table = pd.DataFrame(index=subs, columns=ROIs)

# Loop through subjects and ROIs
for sub in subs:
    for ROI in ROIs:
        fmri_data = get_fmri(sub, ROI)
        # Store the shape (excluding voxel_mask if present)
        shape_table.at[sub, ROI] = fmri_data[0].shape if isinstance(fmri_data, tuple) else fmri_data.shape

# Display the table
print(shape_table)

                V1           V2           V3           V4           LOC  \
sub01  (1000, 232)  (1000, 231)  (1000, 261)  (1000, 107)  (1000, 1843)   
sub02  (1000, 285)  (1000, 231)  (1000, 270)   (1000, 95)  (1000, 1348)   
sub03  (1000, 164)  (1000, 271)  (1000, 270)  (1000, 111)  (1000, 1237)   
sub04  (1000, 176)  (1000, 209)  (1000, 212)  (1000, 117)  (1000, 1153)   
sub05  (1000, 326)  (1000, 196)  (1000, 176)   (1000, 73)  (1000, 1397)   
sub06  (1000, 286)  (1000, 281)  (1000, 229)  (1000, 108)  (1000, 1356)   
sub07  (1000, 195)  (1000, 189)  (1000, 174)   (1000, 55)  (1000, 1117)   
sub08  (1000, 300)  (1000, 238)  (1000, 223)   (1000, 85)  (1000, 1244)   
sub09  (1000, 271)  (1000, 265)  (1000, 245)   (1000, 94)  (1000, 1515)   
sub10  (1000, 238)  (1000, 249)  (1000, 188)   (1000, 60)  (1000, 1034)   

               EBA          FFA          STS          PPA             WB  
sub01  (1000, 351)   (1000, 68)  (1000, 341)  (1000, 425)  (1000, 18222)  
sub02  (1000, 183)  (100

Number of voxels in ROIs differ a lot across subjects.

In [20]:
# Checking for voxel mask in ROI data
ROI_file = os.path.join('participants_data_v2021/mini_track', "sub01/EBA.pkl")
ROI_data = load_dict(ROI_file)
print(ROI_data.keys())
# no voxel mask for ROIs, only for "WB"

dict_keys(['train'])


In [23]:
# do sum of number of voxels of ROI arrays correspond to "WB" array number of voxels?

dimension_table = pd.DataFrame(index=subs, columns=["ROIs_sum", "WB"])

# Loop to compute the dimensions for each subject
for sub in subs:
    sum_second_dim = 0
    wb_second_dim = None

    for ROI in ROIs:
        fmri_data = get_fmri(sub, ROI)
        data = fmri_data[0] if isinstance(fmri_data, tuple) else fmri_data
        if ROI == "WB":
            wb_second_dim = data.shape[1]
        else:
            sum_second_dim += data.shape[1]

    # Store the computed dimensions in the DataFrame
    dimension_table.at[sub, "ROIs_sum"] = sum_second_dim
    dimension_table.at[sub, "WB"] = wb_second_dim

# Display the table
print(dimension_table)


      ROIs_sum     WB
sub01     3859  18222
sub02     3143  21573
sub03     3155  15225
sub04     3068  19445
sub05     2975  13340
sub06     3076  19818
sub07     2033  10836
sub08     2764  12347
sub09     3265  17570
sub10     2285  12950


In [ ]:
# check if after applying voxel mask on WB the number of voxels corresponds to ROIs sum



In [ ]:
# to visualize activity, a voxel mask is needed that maps the 1d ROI arrays into 3d space

from nilearn import plotting
from utils import find_repo_root

def visualize_activity(vid_id, sub, ROI):
    # Setting up the paths for ROI data
    fmri_dir = './participants_data_v2021'
    track = "mini_track" 
    # Get the right track directory
    track_dir = os.path.join(fmri_dir, track)
    sub_fmri_dir = os.path.join(track_dir, sub)

    # Result directory to store NIfTI file
    results_dir = find_repo_root()

    # Mapping the data to voxels and storing in a NIfTI file
    fmri_train = get_fmri(sub_fmri_dir, ROI)

    # Adjusting this part for ROI
    # Assuming fmri_train is now a 2D array of shape (#vids, #voxels)
    fmri_response = fmri_train[vid_id, :]

    # You need a mask or a method to map these voxels back to 3D space
    # Let's assume you have a function to do this mapping
    visual_mask_3D = map_to_3D(fmri_response, ROI)  # This is a hypothetical function

    # Save path for the NIfTI file
    nii_save_path = os.path.join(results_dir, f'{ROI}_activity.nii')

    # Use a brain mask specific to your ROI or a generic brain mask
    brain_mask = './example.nii'  # Update this path to your brain mask
    saveasnii(brain_mask, nii_save_path, visual_mask_3D)

    # Visualizing saved NIfTI file
    plotting.plot_glass_brain(nii_save_path,
                              title=f'fMRI response in {ROI}', plot_abs=False,
                              display_mode='lyr', colorbar=True)
